# Adiabatic SSM model

## Towards closed-loop

This notebook attempts to do finite-horizon predictions using an adiabatic SSM model which has been interpolated from local SSMs on a grid.

In [3]:
from os import listdir
from os.path import join
import pickle
import yaml
import numpy as np
np.set_printoptions(linewidth=100)

In [4]:
%load_ext autoreload
%autoreload 2
import utils as utils
import plot_utils as plot

In [5]:
%matplotlib qt
import matplotlib.pyplot as plt

Load local SSM models

In [1]:
# directory containing the SSM models
model_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic"
model_names = ["northwest", "north", "northeast", "west", "origin", "east", "southwest", "south", "southeast"] # ["north","west", "origin", "east", "south"] # 
N_models = len(model_names)

Load the pre-tensioned equilibrium of each model and plot adiabatic SSM map

In [6]:
q_eq, u_eq = [], []
for model in model_names:
    with open(join(model_dir, model, "SSMmodel", "pre-tensioned_rest_q.pkl"), "rb") as f:
        q_eq.append(pickle.load(f))
    with open(join(model_dir, model, "decay", "info.yaml"), "r") as f:
        info = yaml.safe_load(f)
    u_eq.append(np.array(info['pre_tensioning']))
print(q_eq)
print(u_eq)
xy_rest = np.array([q_eq[i][:2] for i in range(N_models)])

[array([-34.87879219,  35.84572228, -10.60406392]), array([ 0.54467017, 35.57246381, -6.34727727]), array([ 35.552736  ,  34.86109908, -14.7558155 ]), array([-35.30795769,  -0.42439489,  -4.08499071]), array([ 0.03909428, -0.08913968, -0.00175868]), array([34.95441827, -0.27574755, -7.94720868]), array([-35.09507886, -37.14043854, -10.94062807]), array([  0.27917007, -35.77321019,  -6.25407621]), array([ 35.12892494, -35.8256824 , -14.73033781])]
[array([200, 200,   0,   0, 200, 200,   0,   0]), array([200,   0,   0,   0, 200,   0,   0,   0]), array([200,   0,   0, 200, 200,   0,   0, 200]), array([  0, 200,   0,   0,   0, 200,   0,   0]), array([0, 0, 0, 0, 0, 0, 0, 0]), array([  0,   0,   0, 200,   0,   0,   0, 200]), array([  0, 200, 200,   0,   0, 200, 200,   0]), array([  0,   0, 200,   0,   0,   0, 200,   0]), array([  0,   0, 200, 200,   0,   0, 200, 200])]


In [7]:
models = []
for model_name in model_names:
    with open(join(model_dir, model_name, "SSMmodel", "SSM_model.pkl"), "rb") as f:
        model = pickle.load(f)
        models.append(model)
Vde = [model['model']['V'] for model in models]
r_coeff = [model['model']['r_coeff'] for model in models]
w_coeff = [model['model']['w_coeff'] for model in models]
B = [model['model']['B'] for model in models]
ROMOrder = [model['params']['ROM_order'] for model in models]
SSMOrder = [model['params']['SSM_order'] for model in models]

Compute interpolations: Delaunay triangulation, Barycentric linear interpolation

In [8]:
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import CloughTocher2DInterpolator
from scipy.spatial import Delaunay

interpolator = NearestNDInterpolator
# compute delauny triangulation attached to the pre-tensioned equilibria
tri = Delaunay(xy_rest)
# create interpolants for the different coefficient matrices
interpolation = {}
for name, coeffs in [('W', w_coeff), ('V', Vde), ('R', r_coeff), ('B', B), ('u_bar', u_eq), ('q_bar', q_eq)]:
    interpolation[name] = interpolator(tri, coeffs)

Trunk settings

In [9]:
observables = "delay-embedding" # "pos-vel" # 
N_DELAY = 4 # only relevant if observables is "delay-embedding"
TIP_NODE = 51
N_NODES = 709
INPUT_DIM = 8
DT = 0.01

rDOF = 3
oDOF = 3
SSMDim = 6

robot_dir = "../../../soft-robot-control/examples/trunk"
data_dir = "/media/jonas/Backup Plus/jonas_soft_robot_data/trunk_adiabatic/origin"
rest_file = join(robot_dir, 'rest_qv.pkl')

Compute observables (delay embedding)

In [10]:
N_DELAY = 4
assemble_observables = lambda oData: utils.delayEmbedding(oData, up_to_delay=N_DELAY)

Load test trajectory (circle)

In [11]:
test_results = {}
test_trajectory_folders = ['open-loop_circle']
test_trajectories = []
for test_traj in test_trajectory_folders:
    traj_dir = join(robot_dir, "dataCollection", test_traj)
    (t, z), u = utils.import_pos_data(data_dir=traj_dir,
                                      rest_file=rest_file,
                                      output_node=TIP_NODE, return_inputs=True, traj_index=0)
    # z = (z.T - q_eq).T
    # u = (u.T - u_eq).T
    y = assemble_observables(z)
    print("y.shape:", y.shape)
    print("u.shape:", u.shape)

    test_trajectories.append({
            'name': test_traj,
            't': t,
            'z': z,
            'u': u,
            'y': y
        })

y.shape: (15, 1002)
u.shape: (8, 1002)


In [71]:
N_horizon = np.inf

In [72]:
def interpolate_coeffs(xy, coeff_name):
    if coeff_name not in interpolation.keys():
        raise RuntimeError("No interpolation available for these coefficents: {coeff_name}")
    return interpolation[coeff_name](xy).squeeze()

origin_idx = model_names.index("origin")
use_origin_only = False

def advect_adiabaticRD_with_inputs(t, y0, u, y):
    dt = 0.01
    N = len(t)-1
    x = np.full((6, N+1), np.nan)
    y_pred = np.full((15, N+1), np.nan)
    y_pred[:, 0] = y0
    y_bar = np.full((15, N+1), np.nan)
    u_bar = np.full((8, N+1), np.nan)
    xdot = np.full((6, N+1), np.nan)
    weights = np.full((N_models, N+1), np.nan)

    # initialize interpolated model
    y_bar_current, u_bar_current, W_current, R_current, V_current, B_current, weights_current = None, None, None, None, None, None, None

    for i in range(N):
        if i % N_horizon == 0:
            # make observation
            y_pred[:, i] = y[:, i]
            # update model
            if use_origin_only:
                y_bar_current = np.zeros(15)
                u_bar_current = np.zeros(8)
                V_current = Vde[origin_idx]
                R_current = r_coeff[origin_idx]
                W_current = w_coeff[origin_idx]
                B_current = B[origin_idx]
            else:
                y_bar_current = np.tile(interpolate_coeffs(y[:2, i], 'q_bar'), 1 + N_DELAY)
                u_bar_current = interpolate_coeffs(y[:2, i], 'u_bar')
                W_current = interpolate_coeffs(y[:2, i], 'W')
                R_current = interpolate_coeffs(y[:2, i], 'R')
                V_current = interpolate_coeffs(y[:2, i], 'V')
                B_current = interpolate_coeffs(y[:2, i], 'B')
            # x[i] = V[i]^T @ (y[i] - y_bar[i])
            x[:, i] = V_current.T @ (y_pred[:, i].T - y_bar_current) # y[:, i]) # 
            # compute weights used for this finite-horizon rollout
            simplex = tri.find_simplex(y[:2, i])
            b = tri.transform[simplex,:2] @ (y[:2, i] - tri.transform[simplex,2])
            c = np.r_[b, 1 - b.sum()]
            point_idx = tri.simplices[simplex]
            weights_current = np.zeros(N_models)
            if use_origin_only:
                weights_current[origin_idx] = 1
            elif interpolator is NearestNDInterpolator:
                weights_current[point_idx[np.argmax(c)]] = 1
                print(weights_current)
            elif interpolator is LinearNDInterpolator:
                weights_current[point_idx] = c
            else:
                weights_current[:] = np.nan
        try:
            # compute and integrate the dynamics at this timestep
            y_bar[:, i] = y_bar_current
            u_bar[:, i] = u_bar_current
            weights[:, i] = weights_current
            # xdot[i] = R(x[i]) + B[i] @ (u[i] - u_bar[i])
            xdot[:, i] = (R_current @ utils.phi(x[:, i].reshape(-1, 1), ROMOrder[0])).flatten() + B_current @ (u[:, i] - u_bar_current)
            # forward Euler: x[i+1] = x[i] + dt * xdot[i]
            x[:, i+1] = x[:, i] + dt * xdot[:, i]
            # y[i+1] = W(x[i+1]) + y_bar[i]
            y_pred[:, i+1] = (W_current @ utils.phi(x[:, i+1].reshape(-1, 1), SSMOrder[0])).T + y_bar_current
        except Exception as e:
            print("i =", i)
            print("y_pred[i-1]:", y_pred[:, i-1])
            print("y[i-1]:", y[:, i-1])
            print("x[i-1]:", x[:, i-1])
            print("=======================================")
            print("y_pred[i]:", y_pred[:, i])
            print("y[i]:", y[:, i])
            print("x[i]:", x[:, i])
            print("=======================================")
            print("y_pred[i+1]:", y_pred[:, i+1])
            print("y[i+1]:", y[:, i+1])
            print("x[i+1]:", x[:, i+1])
            print("=======================================")
            print(e)
            break
            raise e
    return t, x, y_pred, xdot, y_bar, u_bar, weights

In [73]:
traj = test_trajectories[0]
print(traj['name'])
t, x, y_pred, xdot, y_bar, u_bar, weights = advect_adiabaticRD_with_inputs(traj['t'], traj['y'][:, 0], traj['u'], traj['y'])

open-loop_circle
[0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [74]:
plt.close('all')
# Trajectory xyz vs. t (predicted vs. actual)
axs = plot.traj_xyz_txyz(t,
                            y_pred[0, :], y_pred[1, :], y_pred[2, :],
                            show=False)
axs = plot.traj_xyz_txyz(traj['t'],
                        traj['z'][0, :], traj['z'][1, :], traj['z'][2, :],
                        color="tab:orange", axs=axs, show=False)
axs[-1].legend(["Predicted trajectory", "Actual trajectory"])
# for ax in axs:
#     ax.set_ylim(-35, 35)
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.tight_layout()
plt.show()

# Trajectory 2D (predicted vs. actual)
ax = plot.traj_2D_xy(y_pred[0, :], y_pred[1, :],
                      show=False)
ax = plot.traj_2D_xy(traj['z'][0, :], traj['z'][1, :],
                        color="tab:orange", ax=ax, show=False)
ax.set_xlim(-35, 35)
ax.set_ylim(-35, 35)
ax.legend(["Predicted trajectory", "Actual trajectory"])
plt.suptitle(r"Trajectory (predicted vs. actual)")
plt.show()

# weights
axs = plot.adiabatic_model_weights(t, weights, model_names)

# psi_bar
axs = plot.traj_xyz_txyz(t,
                         y_bar[0, :], y_bar[1, :], y_bar[2, :],
                         xyz_names=[r"$\bar{\psi}_1(t)$", r"$\bar{\psi}_2(t)$", r"$\bar{\psi}_3(t)$"],
                         show=False, rotate_yticks=True)
plt.suptitle(r"$\bar{\psi}(t)$")
plt.show()

# xi_dot
axs = plot.traj_xyz_txyz(t,
                        xdot[0, :], xdot[1, :], xdot[2, :],
                        xyz_names=[r"$\dot{\xi}_1(t)$", r"$\dot{\xi}_2(t)$", r"$\dot{\xi}_3(t)$"],
                        show=False, rotate_yticks=True)
for ax in axs:
    ax.set_ylim(-100, 100)
plt.suptitle(r"$\dot{\xi}(t)$")
plt.show()

# xi
axs = plot.traj_xyz_txyz(t,
                        x[0, :], x[1, :], x[2, :],
                        xyz_names=[r"$\xi_1(t)$", r"$\xi_2(t)$", r"$\xi_3(t)$"],
                        show=False, rotate_yticks=True)
for ax in axs:
    ax.set_ylim(-100, 100)
plt.suptitle(r"$\xi(t)$")
plt.show()
